# Notebook to crossmatch an external catalog with the ALeRCE database

Francisco Förster
2024/01/30

In [7]:
import pandas as pd
import psycopg2
import requests

## Open a connection using the public credentials

In [8]:
credentials_file = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(credentials_file).json()["params"]

In [9]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

## Example external catalog

In [10]:
# The catalog should have id_source, ra, and dec columns
df = pd.read_csv("https://github.com/alercebroker/usecases/blob/master/example_data/watchlist.csv?raw=True")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


In [11]:
def ztf_crossmatch(conn, df, search_radius=1):
    
    '''
    conn: connection to database
    df: external catalog dataframe (with columns id_source, ra, dec)
    search_radius: external radius in arcsec (default=1)
    
    The output is a dataframe with the source id, ra, and dec, 
    as well as the ALeRCE database meanra, meandec, the crossmatch distance 
    in degrees and the time of first detection according to the ALeRCE database
    '''
    
    # fix the dataframe
    objects = []
    for _,row in df.iterrows():
        objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
    objects_str = ",\n".join(objects)
    print(objects_str)
    
    # convert the radius into degrees
    search_radius = search_radius / 3600
    
    # prepare the query

    query = """
    WITH catalog ( source_id, ra, dec) AS (
        VALUES
            {values}
    )
    SELECT 
        c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
        o.firstmjd
    
    FROM object o, catalog c
        /*
         * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
         */
    WHERE
        q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    """
    
    # final query string
    query_str = query.format(values=objects_str, radius=search_radius) # radius in degrees
    
    # do the query
    try:
        matches = pd.read_sql(query_str,conn)
        matches["q3c_dist"] = matches.q3c_dist * 3600
        matches.rename({"q3c_dist": "dist_arcsec"}, axis=1, inplace=True)
        return matches
    except:
        print("Error accessing the database. Most common causes are timeout errors or wrongly formatted input query.")

## Do the crossmatch

Note that the default timeout is 2 min, if your catalog is too large you should split in smaller catalogs and do many queries.

In [12]:
ztf_crossmatch(conn, df)

('source_1', 160.18301441363647, 33.0164673528409),
('source_2', 174.21524897555545, 44.83789535222221)


/home/fforster/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,source_id,ra,dec,oid,meanra,meandec,dist_arcsec,firstmjd
0,source_1,160.183014,33.016467,ZTF18aaacsup,160.182977,33.016294,0.633585,58423.484560
1,source_2,174.215249,44.837895,ZTF18aaapcut,174.215361,44.837997,0.463767,58440.470775


## Close the connection

In [13]:
conn.close()